In [1]:
import sys
import os
import pwd
user = pwd.getpwuid(os.getuid())[0]
sys.path.append(rf"{user}/Flash2OpenPMD/src")
import yt
import numpy as np
import glob
from flash2openpmd import Convert

In [2]:
run_directory = '/media/ong/WORKDIR11/Laser_Tilted_3/'

step = 46

filename = glob.glob1(run_directory,"*_hdf5_plt_cnt_"+str(int(step)).zfill(4))

print(filename[0])

ts = Convert(
    run_directory,
    filename[0],
    # left_edge=[-0.004,0.85e-4,-0.004],
    # right_edge=[0.004,20e-4,0.004],
    )

density = ts.get_data(refinex = 8, refiney = 8) #minimum valueas for x and y

ts.write2openpmd(density)

yt : [INFO     ] 2024-04-15 14:21:25,976 Parameters: current_time              = 2.40033142672008e-10
yt : [INFO     ] 2024-04-15 14:21:25,977 Parameters: domain_dimensions         = [16 16  1]
yt : [INFO     ] 2024-04-15 14:21:25,978 Parameters: domain_left_edge          = [-0.0015  0.      0.    ]
yt : [INFO     ] 2024-04-15 14:21:25,979 Parameters: domain_right_edge         = [0.0015 0.003  1.    ]
yt : [INFO     ] 2024-04-15 14:21:25,980 Parameters: cosmological_simulation   = 0


tilted_hdf5_plt_cnt_0046
level	# grids	       # cells	     # cells^3
----------------------------------------------
  0	     1	           256	             7
  1	     4	          1024	            11
  2	    16	          4096	            16
  3	    64	         16384	            26
  4	   184	         47104	            37
  5	   480	        122880	            50
----------------------------------------------
   	   749	        191744


t = 2.40033143e-10 code_time = 2.40033143e-10 s = 7.60619130e-18 yr

Smallest Cell:
	Width: 1.899e-30 Mpc
	Width: 1.899e-24 pc
	Width: 3.917e-19 AU
	Width: 5.859e-06 cm
    
nx = 256, ny = 256
    
    
n_xnew = 2040, n_ynew = 2040
    
